In [ ]:
import math

# Parameters
Od = 588  # mm, outer diameter
t = 25    # mm, thickness
E = 210000  # MPa, Young's modulus for S355 steel
Fy = 345    # MPa, yield strength for S355 steel
K = 1       # Effective length factor (pinned-pinned)
L = 6288    # mm, length
Nd = 4      # Safety factor

# Section properties
def moment_of_inertia(Od, t):
    Outer_d = Od
    Inner_d = Outer_d - 2 * t
    return (math.pi / 64) * (Outer_d**4 - Inner_d**4)

def radius_of_gyration(Od, t):
    I = moment_of_inertia(Od, t)
    A = (math.pi / 4) * (Od**2 - (Od - 2 * t)**2)
    return math.sqrt(I / A)

I = moment_of_inertia(Od, t)
r = radius_of_gyration(Od, t)
A_s = (math.pi / 4) * (Od**2 - (Od - 2 * t)**2)

print(f"Radius of gyration: {r:.2f} mm")
print(f"Moment of inertia: {I:.2f} mm⁴")

# Buckling load
P_critical = (math.pi ** 2 * E * I) / ((K * L) ** 2)
P_critical_tons = P_critical / 9.80665 / 1000  # N to metric tons

print(f"Critical Buckling Load: {P_critical:.2f} N")
print(f"Critical Buckling Load: {P_critical_tons:.2f} tons")

# Allowable compressive stress (AISC)
def allowable_axial_compressive_stress(K, L, r, Fy, Nd, E):
    KLr = K * L / r
    Cc = math.sqrt((2 * math.pi ** 2 * E) / Fy)
    print(KLr)
    print(Cc)
    if KLr <= Cc:
        Fa = (1 - ((KLr ** 2) / (2 * Cc ** 2))) * (Fy / Nd) * (
            1 + (9 * KLr) / (40 * Cc) - (3 * (KLr ** 3)) / (40 * (Cc ** 4))
        )
    else:
        Fa = (math.pi ** 2 * E) / (1.15 * Nd * (KLr ** 2))
    return Fa

Fa = allowable_axial_compressive_stress(K, L, r, Fy, Nd, E)
Fa_MT = Fa / 10  # 1 MPa = 0.1 MT/m²

print("\nCalculating compressive stress using design code AISC")
print("--" * 35)
print(f"Allowable Axial Compressive Stress: {Fa:.2f} MPa")
print(f"Allowable Axial Compressive Stress: {Fa_MT:.2f} MT/m²")

# Applied compressive force and stress
F = 75.01 * 1000 * 9.80665  # 75.01 T to N
F_app = F / A_s             # MPa
print(F)
print(A_s)
print(F_app)
F_app_MT = F_app / 10       # MT/m²

print(f"Applied Compressive Stress: {F_app:.2f} MPa")
print(f"Applied Compressive Stress: {F_app_MT:.2f} MT/m²")

# Check against allowable
if F_app_MT < Fa_MT:
    print("Applied compressive stress is within allowable limits.")
else:
    print("Applied compressive stress exceeds allowable limits!")


Radius of gyration: 199.25 mm
Moment of inertia: 1755418125.55 mm⁴
Critical Buckling Load: 92018373.48 N
Critical Buckling Load: 9383.26 tons
31.558865461330278
109.61373351564674

Calculating compressive stress using design code AISC
----------------------------------------------------------------------
Allowable Axial Compressive Stress: 88.03 MPa
Allowable Axial Compressive Stress: 8.80 MT/m²
735596.8165
16.63571857458428
44217.91659927634
Applied Compressive Stress: 16.64 MPa
Applied Compressive Stress: 1.66 MT/m²
Applied compressive stress is within allowable limits.
